# Development code for WoofyaCustom
Note: 
- run this file once before WoofyaCustom in a new directory to generate correct input file for WoofyaCustom.

In [1]:
import openai
import pandas as pd
from openai import OpenAI
import random
from haversine import haversine, Unit
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Load the CSV file
file_path = 'recommender_input.csv'
rec_input = pd.read_csv(file_path)

In [3]:
rec_input

,id,place_id,type,detailed_fingerprint,fingerprint,date_updated,date_created,icon,name,scope,...,dog_water,fenced_off,seating,parking,food,night_light,shade,open_field,wildlife,alcohol
0,95,ChIJqWP5r9hp1moRBU5nqhpUHKc,"[""park"", ""point_of_interest"", ""establishment""]",9b045332d6d39862322d10012c1db881,adbe731570a099bd2c393c477363c757,2024-08-25 15:27:42,2024-07-25 15:11:31,https://maps.gstatic.com/mapfiles/place_api/ic...,St Kilda Harbour Dog Beach,NaN,...,0,0,0,0,1,0,0,1,1,0
1,104,ChIJf4fPKjdp1moRueqluvM6rlY,"[""park"", ""point_of_interest"", ""establishment""]",357d9515c4297ae6a56190d36b007a4b,a6d68701d704f67472420331a9587870,2024-09-08 07:54:51,2024-07-25 15:11:31,https://maps.gstatic.com/mapfiles/place_api/ic...,Cummings Reserve,NaN,...,0,1,0,0,0,0,1,1,1,0
2,107,ChIJaSL3-sBn1moRk-T0HL-sYhE,"[""natural_feature"", ""establishment""]",02701ed8df3ed397702bbd638eb43978,be298dc253a97d20accf9ed73176e164,2024-09-08 04:04:58,2024-07-25 15:11:31,https://maps.gstatic.com/mapfiles/place_api/ic...,Port Melbourne Beach,NaN,...,0,0,1,1,1,0,1,0,1,0
3,277,ChIJ0RWwYGdD1moRQHkxBXZWBA8,"[""park"", ""point_of_interest"", ""establishment""]",cebd82bf4dfa4ab2970b306d78f3353e,33ebf94c2adf302959dc0fdd866b35e5,2024-08-25 15:51:51,2024-07-28 13:47:08,https://maps.gstatic.com/mapfiles/place_api/ic...,Phillips Reserve,NaN,...,1,0,1,1,0,0,1,1,1,0
4,290,ChIJsTPuDm1d1moReW-KXiI6Mpg,"[""park"", ""point_of_interest"", ""establishment""]",ad24048d2c5e6e31a8e5134a713d8256,3e4bf53550656e8077b6565a9254c9d8,2024-08-25 15:41:25,2024-07-28 13:47:09,https://maps.gstatic.com/mapfiles/place_api/ic...,Manningham Reserve,NaN,...,0,1,1,1,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,5633,ChIJJWX09g1Z12oR3aGUYn9N-Ls,"[""park"", ""point_of_interest"", ""establishment""]",2fcbba176053b81862b7d377fcde134a,e6e9a2d0feb8c70b53385b0031529bbf,2024-09-08 04:14:00,2024-08-29 14:12:00,https://maps.gstatic.com/mapfiles/place_api/ic...,Strathdale Park Play Space,NaN,...,1,0,1,0,0,0,1,1,0,0
258,5639,ChIJxcNIfetZ12oRY6kCaiNNPk4,"[""park"", ""tourist_attraction"", ""point_of_inter...",44c48fcdb9109512a1cb78e20bb986ad,37503646720eb61578c506fbc6839e7e,2024-09-08 04:14:01,2024-08-29 14:12:01,https://maps.gstatic.com/mapfiles/place_api/ic...,Rosalind Park,NaN,...,0,0,1,0,1,1,1,0,1,0
259,5641,ChIJj6zUo29X12oR7BPNSN_Eo20,"[""park"", ""point_of_interest"", ""establishment""]",0bb3cc82f97399eab5998a99ba7310e7,3ce985a1e48e1e6bdfa12b36b2da481e,2024-09-08 07:57:29,2024-08-29 14:12:03,https://maps.gstatic.com/mapfiles/place_api/ic...,Truscott Reserve Dog Park,NaN,...,1,1,1,1,0,0,1,1,0,0
260,5656,ChIJX2Nmf3pZ12oRMK1ntODuWvQ,"[""park"", ""health"", ""point_of_interest"", ""estab...",9c808a9b705c1d225360275f945f7f6e,ddf5824ee904324ddf0443a1e93a2c6f,2024-09-08 04:14:02,2024-08-29 14:12:03,https://maps.gstatic.com/mapfiles/place_api/ic...,Kennington Reservoir parkrun,NaN,...,1,1,1,1,0,0,1,1,1,0


In [4]:
input_select = rec_input[['name', 'suburb', 'rating', 'vicinity', 'review_text', 'type', 'editorial_summary.overview', 'description', 'wheelchair_accessible_entrance', 'food', 'serves_vegetarian_food', 'off_leash', 'dog_water', 'fenced_off', 'seating', 'parking', 'night_light', 'shade', 'wildlife', 'open_field', 'alcohol', 'geometry.location.lat', 'geometry.location.lng']]
input_select

,name,suburb,rating,vicinity,review_text,type,editorial_summary.overview,description,wheelchair_accessible_entrance,food,...,fenced_off,seating,parking,night_light,shade,wildlife,open_field,alcohol,geometry.location.lat,geometry.location.lng
0,St Kilda Harbour Dog Beach,St Kilda,4.9,NaN,['Small area of beach just north of the main p...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Dog-friendly beach', 'Scenic views of the ha...",NaN,1,...,0,0,0,0,0,1,1,0,-37.860646,144.969802
1,Cummings Reserve,St Kilda West,4.6,"Cummings Reserve, 299-312 Beaconsfield Parade,...","[""Cummings Reserve is quite a large tapering s...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Grassed Area', 'Scenic Sunset Views', 'Dog W...",True,0,...,1,0,0,0,1,1,1,0,-37.856424,144.965946
2,Port Melbourne Beach,Port Melbourne,4.5,Port Melbourne Beach,"['Stunning beauty, You can try fishing at the ...","[""natural_feature"", ""establishment""]",Family-friendly beach with a small wooden fish...,"['Stunning beauty', 'Family-friendly', 'Gentle...",NaN,1,...,0,1,1,0,1,1,0,0,-37.847039,144.945525
3,Phillips Reserve,Brunswick East,4.5,Brunswick East,['Council should maintain there pathways they ...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-friendly playground', 'Tranquil natur...",True,0,...,0,1,1,0,1,1,1,0,-37.770638,144.984935
4,Manningham Reserve,Parkville,4.6,"Substation 100/87 Manningham Street, Parkville","[""Nice and clean, it's connected to the soccer...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-Friendly', 'Sports Activities', 'Gree...",True,0,...,1,1,1,0,1,1,1,0,-37.785144,144.941920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,Strathdale Park Play Space,Kennington,4.6,"148 Crook Street, Kennington",['Beautiful space for everyone to enjoy.\nLove...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-friendly', 'Playgrounds', 'Water feat...",True,0,...,0,1,0,0,1,0,1,0,-36.760569,144.313850
258,Rosalind Park,Bendigo,4.6,"30 Gaol Road, Bendigo",['Very nice leafy park with gardens and a cons...,"[""park"", ""tourist_attraction"", ""point_of_inter...","Downtown park with a fernery, an 1800s conserv...","['Leafy park with gardens and conservatory', '...",True,1,...,0,1,0,1,1,1,0,0,-36.757165,144.278365
259,Truscott Reserve Dog Park,California Gully,4.3,"Cnr of Murdock St and, Turner Street, Californ...","[""Nice big park, there's a small dog area aswe...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Spacious environment for dogs', 'Separate ar...",True,0,...,1,1,1,0,1,0,1,0,-36.728133,144.252898
260,Kennington Reservoir parkrun,Strathdale,4.4,"55 Reservoir Road, Strathdale","['I really like this parkrun, although the dou...","[""park"", ""health"", ""point_of_interest"", ""estab...",NaN,"['Scenic Reservoir Views', 'Friendly Community...",True,0,...,1,1,1,0,1,1,1,0,-36.772869,144.311283


In [5]:
input_select['wheelchair_accessible_entrance'].value_counts()

True     208
False      5
Name: wheelchair_accessible_entrance, dtype: int64

In [6]:
input_select['wheelchair_accessible_entrance'].fillna(0, inplace=True)
input_select['wheelchair_accessible_entrance'].replace({True:1, False:0}, inplace=True)
input_select['wheelchair_accessible_entrance'].value_counts()

C:\Users\liamg\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\liamg\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


1    208
0     54
Name: wheelchair_accessible_entrance, dtype: int64

In [7]:
input_select['serves_vegetarian_food'].value_counts()

True    22
Name: serves_vegetarian_food, dtype: int64

In [8]:
input_select['serves_vegetarian_food'].fillna(0, inplace=True)
input_select['serves_vegetarian_food'].replace({True:1, False:0}, inplace=True)
input_select['serves_vegetarian_food'].value_counts()

0    240
1     22
Name: serves_vegetarian_food, dtype: int64

In [9]:
input_select

,name,suburb,rating,vicinity,review_text,type,editorial_summary.overview,description,wheelchair_accessible_entrance,food,...,fenced_off,seating,parking,night_light,shade,wildlife,open_field,alcohol,geometry.location.lat,geometry.location.lng
0,St Kilda Harbour Dog Beach,St Kilda,4.9,NaN,['Small area of beach just north of the main p...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Dog-friendly beach', 'Scenic views of the ha...",0,1,...,0,0,0,0,0,1,1,0,-37.860646,144.969802
1,Cummings Reserve,St Kilda West,4.6,"Cummings Reserve, 299-312 Beaconsfield Parade,...","[""Cummings Reserve is quite a large tapering s...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Grassed Area', 'Scenic Sunset Views', 'Dog W...",1,0,...,1,0,0,0,1,1,1,0,-37.856424,144.965946
2,Port Melbourne Beach,Port Melbourne,4.5,Port Melbourne Beach,"['Stunning beauty, You can try fishing at the ...","[""natural_feature"", ""establishment""]",Family-friendly beach with a small wooden fish...,"['Stunning beauty', 'Family-friendly', 'Gentle...",0,1,...,0,1,1,0,1,1,0,0,-37.847039,144.945525
3,Phillips Reserve,Brunswick East,4.5,Brunswick East,['Council should maintain there pathways they ...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-friendly playground', 'Tranquil natur...",1,0,...,0,1,1,0,1,1,1,0,-37.770638,144.984935
4,Manningham Reserve,Parkville,4.6,"Substation 100/87 Manningham Street, Parkville","[""Nice and clean, it's connected to the soccer...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-Friendly', 'Sports Activities', 'Gree...",1,0,...,1,1,1,0,1,1,1,0,-37.785144,144.941920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,Strathdale Park Play Space,Kennington,4.6,"148 Crook Street, Kennington",['Beautiful space for everyone to enjoy.\nLove...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-friendly', 'Playgrounds', 'Water feat...",1,0,...,0,1,0,0,1,0,1,0,-36.760569,144.313850
258,Rosalind Park,Bendigo,4.6,"30 Gaol Road, Bendigo",['Very nice leafy park with gardens and a cons...,"[""park"", ""tourist_attraction"", ""point_of_inter...","Downtown park with a fernery, an 1800s conserv...","['Leafy park with gardens and conservatory', '...",1,1,...,0,1,0,1,1,1,0,0,-36.757165,144.278365
259,Truscott Reserve Dog Park,California Gully,4.3,"Cnr of Murdock St and, Turner Street, Californ...","[""Nice big park, there's a small dog area aswe...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Spacious environment for dogs', 'Separate ar...",1,0,...,1,1,1,0,1,0,1,0,-36.728133,144.252898
260,Kennington Reservoir parkrun,Strathdale,4.4,"55 Reservoir Road, Strathdale","['I really like this parkrun, although the dou...","[""park"", ""health"", ""point_of_interest"", ""estab...",NaN,"['Scenic Reservoir Views', 'Friendly Community...",1,0,...,1,1,1,0,1,1,1,0,-36.772869,144.311283


In [10]:
input_select.to_csv('input_select.csv', index=False)

In [11]:
# Distance Filter
def calulate_dist(row, user_lat, user_lon):
    location_point = (row['geometry.location.lat'], row['geometry.location.lng'])
    user_point = (user_lat, user_lon)
    return haversine(user_point, location_point, unit=Unit.KILOMETERS)

In [12]:
# Recommendation Function version 1:
#- basic recommender based on user location only.

def recommender_v1(user_lat, user_lon, input_select, f_dist):

    input_select['h_dist'] = input_select.apply(calulate_dist, axis=1, user_lat=user_lat, user_lon=user_lon)
    filtered_locs = input_select[input_select['h_dist'] <= f_dist]
    return filtered_locs

In [13]:
input_select

,name,suburb,rating,vicinity,review_text,type,editorial_summary.overview,description,wheelchair_accessible_entrance,food,...,fenced_off,seating,parking,night_light,shade,wildlife,open_field,alcohol,geometry.location.lat,geometry.location.lng
0,St Kilda Harbour Dog Beach,St Kilda,4.9,NaN,['Small area of beach just north of the main p...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Dog-friendly beach', 'Scenic views of the ha...",0,1,...,0,0,0,0,0,1,1,0,-37.860646,144.969802
1,Cummings Reserve,St Kilda West,4.6,"Cummings Reserve, 299-312 Beaconsfield Parade,...","[""Cummings Reserve is quite a large tapering s...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Grassed Area', 'Scenic Sunset Views', 'Dog W...",1,0,...,1,0,0,0,1,1,1,0,-37.856424,144.965946
2,Port Melbourne Beach,Port Melbourne,4.5,Port Melbourne Beach,"['Stunning beauty, You can try fishing at the ...","[""natural_feature"", ""establishment""]",Family-friendly beach with a small wooden fish...,"['Stunning beauty', 'Family-friendly', 'Gentle...",0,1,...,0,1,1,0,1,1,0,0,-37.847039,144.945525
3,Phillips Reserve,Brunswick East,4.5,Brunswick East,['Council should maintain there pathways they ...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-friendly playground', 'Tranquil natur...",1,0,...,0,1,1,0,1,1,1,0,-37.770638,144.984935
4,Manningham Reserve,Parkville,4.6,"Substation 100/87 Manningham Street, Parkville","[""Nice and clean, it's connected to the soccer...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-Friendly', 'Sports Activities', 'Gree...",1,0,...,1,1,1,0,1,1,1,0,-37.785144,144.941920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,Strathdale Park Play Space,Kennington,4.6,"148 Crook Street, Kennington",['Beautiful space for everyone to enjoy.\nLove...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-friendly', 'Playgrounds', 'Water feat...",1,0,...,0,1,0,0,1,0,1,0,-36.760569,144.313850
258,Rosalind Park,Bendigo,4.6,"30 Gaol Road, Bendigo",['Very nice leafy park with gardens and a cons...,"[""park"", ""tourist_attraction"", ""point_of_inter...","Downtown park with a fernery, an 1800s conserv...","['Leafy park with gardens and conservatory', '...",1,1,...,0,1,0,1,1,1,0,0,-36.757165,144.278365
259,Truscott Reserve Dog Park,California Gully,4.3,"Cnr of Murdock St and, Turner Street, Californ...","[""Nice big park, there's a small dog area aswe...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Spacious environment for dogs', 'Separate ar...",1,0,...,1,1,1,0,1,0,1,0,-36.728133,144.252898
260,Kennington Reservoir parkrun,Strathdale,4.4,"55 Reservoir Road, Strathdale","['I really like this parkrun, although the dou...","[""park"", ""health"", ""point_of_interest"", ""estab...",NaN,"['Scenic Reservoir Views', 'Friendly Community...",1,0,...,1,1,1,0,1,1,1,0,-36.772869,144.311283


In [14]:
locations = recommender_v1(-37.847039, 144.945525, input_select, 10)

C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\2556746437.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['h_dist'] = input_select.apply(calulate_dist, axis=1, user_lat=user_lat, user_lon=user_lon)


In [15]:
locations

,name,suburb,rating,vicinity,review_text,type,editorial_summary.overview,description,wheelchair_accessible_entrance,food,...,seating,parking,night_light,shade,wildlife,open_field,alcohol,geometry.location.lat,geometry.location.lng,h_dist
0,St Kilda Harbour Dog Beach,St Kilda,4.9,NaN,['Small area of beach just north of the main p...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Dog-friendly beach', 'Scenic views of the ha...",0,1,...,0,0,0,0,1,1,0,-37.860646,144.969802,2.613904
1,Cummings Reserve,St Kilda West,4.6,"Cummings Reserve, 299-312 Beaconsfield Parade,...","[""Cummings Reserve is quite a large tapering s...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Grassed Area', 'Scenic Sunset Views', 'Dog W...",1,0,...,0,0,0,1,1,1,0,-37.856424,144.965946,2.074537
2,Port Melbourne Beach,Port Melbourne,4.5,Port Melbourne Beach,"['Stunning beauty, You can try fishing at the ...","[""natural_feature"", ""establishment""]",Family-friendly beach with a small wooden fish...,"['Stunning beauty', 'Family-friendly', 'Gentle...",0,1,...,1,1,0,1,1,0,0,-37.847039,144.945525,0.000028
3,Phillips Reserve,Brunswick East,4.5,Brunswick East,['Council should maintain there pathways they ...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-friendly playground', 'Tranquil natur...",1,0,...,1,1,0,1,1,1,0,-37.770638,144.984935,9.173807
4,Manningham Reserve,Parkville,4.6,"Substation 100/87 Manningham Street, Parkville","[""Nice and clean, it's connected to the soccer...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-Friendly', 'Sports Activities', 'Gree...",1,0,...,1,1,0,1,1,1,0,-37.785144,144.941920,6.889736
5,The Stables of Como,South Yarra,3.6,"Williams Rd &, Lechlade Avenue, South Yarra","[""very interesting location, great spot to com...","[""cafe"", ""food"", ""point_of_interest"", ""establi...",Quaint shed in the grounds of an upscale count...,"['Stunning gardens', 'Reasonably priced food',...",0,1,...,1,1,0,1,0,1,1,-37.837824,145.003409,5.185078
6,Jardin Tan,South Yarra,3.7,"Royal Botanic Gardens, Birdwood Avenue, South ...",['Had a quick stop to warm ourselves up before...,"[""cafe"", ""restaurant"", ""food"", ""point_of_inter...",NaN,"['casual dining', 'family-friendly', 'beautifu...",1,1,...,1,0,1,1,0,0,1,-37.829634,144.975774,3.286636
7,Apollo and Friends Cafe,West Melbourne,4.5,"109-111 Hawke Street, West Melbourne","[""I really enjoyed my experience and food. The...","[""cafe"", ""point_of_interest"", ""food"", ""establi...",NaN,"['cozy atmosphere', 'friendly staff', 'fresh h...",1,1,...,1,0,1,0,0,0,0,-37.807822,144.946572,4.361684
8,Balderdash,Port Melbourne,4.4,"295 Bay Street, Port Melbourne",['We had a lovely time with our toddler. He lo...,"[""cafe"", ""store"", ""food"", ""point_of_interest"",...",Corner spot with a cool vibe offering specialt...,"['Family-friendly', 'Cozy atmosphere', 'Specia...",0,1,...,1,0,0,0,0,0,0,-37.836944,144.943611,1.134982
9,Be The Duck,Richmond,4.8,"131 Burnley Street, Richmond","[""I can confidently say that Chris the owners ...","[""cafe"", ""meal_takeaway"", ""store"", ""restaurant...",NaN,"['Cozy atmosphere', 'Passionate owner', 'Outst...",1,1,...,1,1,0,0,0,0,0,-37.816529,145.009065,6.530643


In [16]:
# Recommendation Function version 2:
#- basic recommender based on user location filtering, as well as binary wheelchair and vegetarian filters. Need to loosen the 
# strictness of the wc and veg filters to allow no preference.

def recommender_v2(user_lat, user_lon, input_select, f_dist, wheelchair_accessible_entrance, serves_vegetarian_food):

    input_select['h_dist'] = input_select.apply(calulate_dist, axis=1, user_lat=user_lat, user_lon=user_lon)
    filtered_locs = input_select[
        (input_select['h_dist'] <= f_dist) &
        (input_select['wheelchair_accessible_entrance'] == wheelchair_accessible_entrance) &
        (input_select['serves_vegetarian_food'] == serves_vegetarian_food)
    ]
    return filtered_locs

In [17]:
locations = recommender_v2(-37.847039, 144.945525, input_select, 10, 1, 0)

C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\3141698864.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['h_dist'] = input_select.apply(calulate_dist, axis=1, user_lat=user_lat, user_lon=user_lon)


In [18]:
locations

,name,suburb,rating,vicinity,review_text,type,editorial_summary.overview,description,wheelchair_accessible_entrance,food,...,seating,parking,night_light,shade,wildlife,open_field,alcohol,geometry.location.lat,geometry.location.lng,h_dist
1,Cummings Reserve,St Kilda West,4.6,"Cummings Reserve, 299-312 Beaconsfield Parade,...","[""Cummings Reserve is quite a large tapering s...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Grassed Area', 'Scenic Sunset Views', 'Dog W...",1,0,...,0,0,0,1,1,1,0,-37.856424,144.965946,2.074537
3,Phillips Reserve,Brunswick East,4.5,Brunswick East,['Council should maintain there pathways they ...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-friendly playground', 'Tranquil natur...",1,0,...,1,1,0,1,1,1,0,-37.770638,144.984935,9.173807
4,Manningham Reserve,Parkville,4.6,"Substation 100/87 Manningham Street, Parkville","[""Nice and clean, it's connected to the soccer...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-Friendly', 'Sports Activities', 'Gree...",1,0,...,1,1,0,1,1,1,0,-37.785144,144.941920,6.889736
9,Be The Duck,Richmond,4.8,"131 Burnley Street, Richmond","[""I can confidently say that Chris the owners ...","[""cafe"", ""meal_takeaway"", ""store"", ""restaurant...",NaN,"['Cozy atmosphere', 'Passionate owner', 'Outst...",1,1,...,1,1,0,0,0,0,0,-37.816529,145.009065,6.530643
10,Bell Street Coffee Window,Fitzroy,4.4,"233 Brunswick Street, Fitzroy","[""To my taste, the coffee was burnt, not my fa...","[""cafe"", ""bar"", ""food"", ""point_of_interest"", ""...",NaN,"['Coffee Quality', 'Service Experience', 'Comm...",1,1,...,1,0,0,1,0,0,0,-37.800440,144.977879,5.909643
17,Café Felice,Melbourne,4.6,"461 Bourke Street, Melbourne","[""Went in here on my way to work because the t...","[""cafe"", ""food"", ""point_of_interest"", ""store"",...",NaN,"['Best coffee in Melbourne CBD', 'Friendly and...",1,1,...,1,0,0,1,0,0,0,-37.815553,144.960237,3.731898
21,Caffe Di Lido,Port Melbourne,4.3,"40 Beaconsfield Parade, Port Melbourne",['Often the turnaround point for bike rides al...,"[""cafe"", ""food"", ""store"", ""point_of_interest"",...",NaN,"['Beachfront Location', 'Friendly Atmosphere',...",1,1,...,1,0,1,1,1,1,0,-37.845023,144.942102,0.374938
24,The Ascot Cafe,Ascot Vale,4.2,"197 Union Road, Ascot Vale",['I’m a very regular visitor to cafes across M...,"[""cafe"", ""restaurant"", ""food"", ""point_of_inter...",NaN,"['Great Coffee', 'Friendly Atmosphere', 'Tasty...",1,1,...,1,0,0,0,0,0,0,-37.773087,144.915716,8.630003
35,Fed,Albert Park,4.7,"131 Dundas Place, Albert Park",['Went here for a quick lunch. It was a little...,"[""restaurant"", ""cafe"", ""store"", ""food"", ""point...",NaN,"['Friendly staff', 'Great coffee', 'Delicious ...",1,1,...,1,0,0,0,0,0,0,-37.841341,144.953938,0.973240
38,Harleston Park Dog Off Leash Area,Elsternwick,3.7,"40 Allison Road, Elsternwick","[""I had the pleasure of dog-sitting a friend's...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Dog-friendly', 'Family-oriented', 'Outdoor a...",1,0,...,1,1,0,1,1,1,0,-37.881655,145.010204,6.859588


In [19]:
# Recommendation Function version 3:
#- basic recommender based on user location filtering, as well as binary features - ooleash, drinking water, fenced off
# seating available, parking available, wheelchair access, serving food/veg food, night lighting, shade, open-field, wildlife. 
# These are categories that I am confident that GPT is correct when confirming presence of such features. However, due to the 
# nature of GPT, I recommend '0' be interpreted as 'unconfirmed' as opposed to 'no'.

def recommender_v3(user_lat, user_lon, input_select, f_dist, wheelchair_accessible_entrance, food, serves_vegetarian_food, off_leash, dog_water, fenced_off, seating, parking, night_light, shade, wildlife, open_field, alcohol):

    input_select['h_dist'] = input_select.apply(calulate_dist, axis=1, user_lat=user_lat, user_lon=user_lon)
    filtered_locs = input_select[
        (input_select['h_dist'] <= f_dist) &
        (input_select['wheelchair_accessible_entrance'] == wheelchair_accessible_entrance) &
        (input_select['serves_vegetarian_food'] == serves_vegetarian_food) &
        (input_select['off_leash'] == off_leash) &
        (input_select['dog_water'] == dog_water) &
        (input_select['fenced_off'] == fenced_off) &
        (input_select['seating'] == seating) & 
        (input_select['parking'] == parking) &
        (input_select['food'] == food) &
        (input_select['night_light'] == night_light) &
        (input_select['shade'] == shade) &
        (input_select['open_field'] == open_field) & 
        (input_select['wildlife'] == wildlife) & 
        (input_select['alcohol'] == alcohol)
    ]
    return filtered_locs

In [20]:
locations = recommender_v3(-37.847039, 144.945525, input_select, 100, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0)

C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\2178009535.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['h_dist'] = input_select.apply(calulate_dist, axis=1, user_lat=user_lat, user_lon=user_lon)


In [21]:
locations

,name,suburb,rating,vicinity,review_text,type,editorial_summary.overview,description,wheelchair_accessible_entrance,food,...,seating,parking,night_light,shade,wildlife,open_field,alcohol,geometry.location.lat,geometry.location.lng,h_dist
142,Spring Road Reserve Off-leash Dog Park,Dingley Village,4.5,"241 Spring Road, Dingley Village",['Great dog park. Has the fully fenced area th...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Fenced Area', 'Large Off-leash Space', 'Wate...",0,0,...,0,0,0,1,1,1,0,-37.973999,145.133623,21.716581
152,Emerson Place Reserve Dog Off Leash Area,Rowville,4.4,"17 Emerson Place, Rowville",['This would have to be one of the best doggie...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['dog-friendly', 'enclosed areas', 'well-maint...",0,0,...,0,0,0,1,1,1,0,-37.929425,145.250157,28.259401


### recommender_v4
- binary preferences now only exclude locations when a feature is not present (i.e. 'food' = 0) when a user states it is
  required (i.e. user_input['food'] = 1).
- location filtering still excludes all options not within stated distance. However it does include a proximity score.

In [22]:
def recommender_v4(user_lat, user_lon, input_select, f_dist, wheelchair_accessible_entrance, food, serves_vegetarian_food, off_leash, dog_water, fenced_off, seating, parking, night_light, shade, wildlife, open_field, alcohol):

    filtered_locs = pd.DataFrame()
    input_select['h_dist'] = input_select.apply(calulate_dist, axis=1, user_lat=user_lat, user_lon=user_lon)
    input_select['prox_score'] = input_select['h_dist'].apply(lambda d: max(0, 1-d/f_dist) if f_dist > 0 else 0)
    user_reqs = {
        'wheelchair_accessible_entrance': wheelchair_accessible_entrance,
        'food': food, 
        'serves_vegetarian_food': serves_vegetarian_food,
        'off_leash': off_leash, 
        'dog_water': dog_water, 
        'fenced_off': fenced_off, 
        'seating': seating, 
        'parking': parking, 
        'night_light': night_light, 
        'shade': shade, 
        'wildlife': wildlife, 
        'open_field': open_field, 
        'alcohol': alcohol
    }
    mask = pd.Series([True] * len(input_select))
    for col, req in user_reqs.items():
        if req == 1:
            mask &= input_select[col] == 1
    filtered_locs = input_select[mask]
    filtered_locs = filtered_locs[filtered_locs['h_dist'] <= f_dist]
    return filtered_locs

In [23]:
locations = recommender_v4(-37.847039, 144.945525, input_select, 50, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\3672555081.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['h_dist'] = input_select.apply(calulate_dist, axis=1, user_lat=user_lat, user_lon=user_lon)
C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\3672555081.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['prox_score'] = input_select['h_dist'].apply(lambda d: max(0, 1-d/f_dist) if f_dist > 0 else 0)


In [24]:
locations

,name,suburb,rating,vicinity,review_text,type,editorial_summary.overview,description,wheelchair_accessible_entrance,food,...,parking,night_light,shade,wildlife,open_field,alcohol,geometry.location.lat,geometry.location.lng,h_dist,prox_score
0,St Kilda Harbour Dog Beach,St Kilda,4.9,NaN,['Small area of beach just north of the main p...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Dog-friendly beach', 'Scenic views of the ha...",0,1,...,0,0,0,1,1,0,-37.860646,144.969802,2.613904,0.947722
1,Cummings Reserve,St Kilda West,4.6,"Cummings Reserve, 299-312 Beaconsfield Parade,...","[""Cummings Reserve is quite a large tapering s...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Grassed Area', 'Scenic Sunset Views', 'Dog W...",1,0,...,0,0,1,1,1,0,-37.856424,144.965946,2.074537,0.958509
2,Port Melbourne Beach,Port Melbourne,4.5,Port Melbourne Beach,"['Stunning beauty, You can try fishing at the ...","[""natural_feature"", ""establishment""]",Family-friendly beach with a small wooden fish...,"['Stunning beauty', 'Family-friendly', 'Gentle...",0,1,...,1,0,1,1,0,0,-37.847039,144.945525,0.000028,0.999999
3,Phillips Reserve,Brunswick East,4.5,Brunswick East,['Council should maintain there pathways they ...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-friendly playground', 'Tranquil natur...",1,0,...,1,0,1,1,1,0,-37.770638,144.984935,9.173807,0.816524
4,Manningham Reserve,Parkville,4.6,"Substation 100/87 Manningham Street, Parkville","[""Nice and clean, it's connected to the soccer...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-Friendly', 'Sports Activities', 'Gree...",1,0,...,1,0,1,1,1,0,-37.785144,144.941920,6.889736,0.862205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Lerderderg Walking Track,Darley,4.4,"6 Janette Court, Darley",['A peaceful trail to walk on a Weekend where ...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Scenic River Views', 'Birdwatching Paradise'...",0,0,...,0,0,1,1,0,0,-37.646819,144.438987,49.791643,0.004167
221,Stonehill Park,Maddingley,4.3,Maddingley,"[""Please council people wash or replace the st...","[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Stunning Views', 'Family-Friendly Play Areas...",1,0,...,0,0,1,0,1,0,-37.685260,144.419020,49.653684,0.006926
241,Sunbury Dog Park,Sunbury,4.5,"69 Harker Street, Sunbury",['Lovely off leash dog and picnic area. A ch...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Off-leash area', 'Shade and seating', 'Agili...",1,0,...,1,0,1,0,1,0,-37.585921,144.733266,34.519799,0.309604
242,The Nook,Sunbury,4.6,"Vaughan Street, Sunbury",['Absolutely lovely little spot to take the ki...,"[""park"", ""point_of_interest"", ""establishment""]",NaN,"['Family-friendly', 'Natural beauty', 'Dog-fri...",1,0,...,0,0,1,1,1,0,-37.580503,144.739915,34.720232,0.305595


### recommender_v5:
- Binary category filtering options
- Exclude locations outside max distance
- Key word search for user
- Weighting options to balance importance of distance vs key word

In [25]:
# Initialise stemmer and stop words, then create preprocessing function
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
def search_preprocessing(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = [stemmer.stem(word) for word in text.split() if word not in stop_words]
    return tokens

In [26]:
keyword_index = {}
for idx, row in input_select.iterrows():
    combined_text = f"{row['name']} {row['description']} {row['review_text']} {row['editorial_summary.overview']}"
    tokens = search_preprocessing(combined_text)
    
    for token in tokens:
        if token not in keyword_index:
            keyword_index[token] = []
        keyword_index[token].append(idx)
        
print(keyword_index)

{'st': [0, 0, 0, 0, 0, 1, 2, 10, 10, 10, 19, 30, 41, 130, 205, 217, 231, 255], 'kilda': [0, 0, 0, 0, 0, 0, 0, 1, 2, 30, 130], 'harbour': [0, 0, 0, 0, 0], 'dog': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 3, 3, 4, 10, 10, 15, 15, 15, 30, 34, 34, 38, 38, 38, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 40, 40, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 45, 45, 45, 45, 45, 45, 45, 46, 46, 47, 47, 47, 50, 50, 52, 52, 53, 54, 54, 54, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 59, 59, 59, 59, 59, 59, 59, 60, 60, 60, 60, 60, 60, 60, 60, 60, 61, 61, 61, 61, 61, 61, 61, 62, 62, 62, 62, 62, 62, 63, 63, 63, 63, 63, 64, 64, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 68, 68, 68, 68, 68, 68, 68, 69, 69, 69, 69, 69, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 74, 74, 74, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 7

In [27]:
combined_text

"Moama Off Leash Dog Park ['Clean and well-maintained', 'Spacious and fully fenced', 'Variety of terrains for exploration', 'Separate area for small or anxious dogs', 'Relaxing environment with plenty of seating'] ['One of the cleanest and largest dog parks we have used. Plenty of areas where dogs can climb, dig, sniff and play. Great seating for dog owners and plenty of fresh water. Fully fenced with a smaller fenced area for little, older and timid dogs 🐕.', 'Excellent dog park - best I’ve ever seen. Huge, fully fenced area. Lots of different types of environments for them to explore. Also has a separate area for small or anxious dogs.', 'Just wonderful. Neat, tidy, clean, shade and sun and a variety of surfaces and terrains, interesting for hoomans and pups. Everyone we met today was lovely. Will be back...prob very soon!', 'One of the biggest and best parks\\nLots of the dogs to sniff and jump on', 'Lots of room for our dog to run. Gardens for her to sniff around in. Plenty of wate

In [28]:
# Create function to search key word
def keyword_search(input_select, keyword):
    kw_tokens = search_preprocessing(keyword)
    result_inds = set()
    
    for token in kw_tokens:
        if token in kw_tokens:
            result_inds.update(keyword_index[token])
    
    input_select['name'] = input_select['name'].fillna('')
    input_select['description'] = input_select['description'].fillna('')
    input_select['review_text'] = input_select['review_text'].fillna('')
    input_select['editorial_summary.overview'] = input_select['editorial_summary.overview'].fillna('')
    input_select['full_text'] = input_select['name'] + ' ' + input_select['description'] + ' ' + input_select['review_text'] + ' ' + input_select['editorial_summary.overview']
    input_select['full_text'] = input_select['full_text'].fillna('')
    results = input_select
   
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(results['full_text'].tolist() + [keyword])
    
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    results['cosine_similarity'] = cosine_similarities.flatten()
    
    return results.iloc[list(result_inds)]

In [29]:
results = keyword_search(input_select, 'dog')

C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['name'] = input_select['name'].fillna('')
C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['description'] = input_select['description'].fillna('')
C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [30]:
results

,name,suburb,rating,vicinity,review_text,type,editorial_summary.overview,description,wheelchair_accessible_entrance,food,...,shade,wildlife,open_field,alcohol,geometry.location.lat,geometry.location.lng,h_dist,prox_score,full_text,cosine_similarity
0,St Kilda Harbour Dog Beach,St Kilda,4.9,NaN,['Small area of beach just north of the main p...,"[""park"", ""point_of_interest"", ""establishment""]",,"['Dog-friendly beach', 'Scenic views of the ha...",0,1,...,0,1,1,0,-37.860646,144.969802,2.613904,0.947722,St Kilda Harbour Dog Beach ['Dog-friendly beac...,0.125715
1,Cummings Reserve,St Kilda West,4.6,"Cummings Reserve, 299-312 Beaconsfield Parade,...","[""Cummings Reserve is quite a large tapering s...","[""park"", ""point_of_interest"", ""establishment""]",,"['Grassed Area', 'Scenic Sunset Views', 'Dog W...",1,0,...,1,1,1,0,-37.856424,144.965946,2.074537,0.958509,"Cummings Reserve ['Grassed Area', 'Scenic Suns...",0.056770
3,Phillips Reserve,Brunswick East,4.5,Brunswick East,['Council should maintain there pathways they ...,"[""park"", ""point_of_interest"", ""establishment""]",,"['Family-friendly playground', 'Tranquil natur...",1,0,...,1,1,1,0,-37.770638,144.984935,9.173807,0.816524,Phillips Reserve ['Family-friendly playground'...,0.058575
4,Manningham Reserve,Parkville,4.6,"Substation 100/87 Manningham Street, Parkville","[""Nice and clean, it's connected to the soccer...","[""park"", ""point_of_interest"", ""establishment""]",,"['Family-Friendly', 'Sports Activities', 'Gree...",1,0,...,1,1,1,0,-37.785144,144.941920,6.889736,0.862205,"Manningham Reserve ['Family-Friendly', 'Sports...",0.090088
10,Bell Street Coffee Window,Fitzroy,4.4,"233 Brunswick Street, Fitzroy","[""To my taste, the coffee was burnt, not my fa...","[""cafe"", ""bar"", ""food"", ""point_of_interest"", ""...",,"['Coffee Quality', 'Service Experience', 'Comm...",1,1,...,1,0,0,0,-37.800440,144.977879,5.909643,0.881807,"Bell Street Coffee Window ['Coffee Quality', '...",0.029724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Mildura Fenced Dog Park (Rio Vista Park),Mildura,4.2,"Hugh King Drive, Mildura","[""Awesome place to let your dog off lead, the ...","[""park"", ""point_of_interest"", ""establishment""]",,"['Spacious and well-maintained dog park', 'Sha...",1,0,...,1,1,1,0,-34.175648,142.160300,478.915938,0.000000,Mildura Fenced Dog Park (Rio Vista Park) ['Spa...,0.197250
254,Allingham Reserve Dog Park,Kangaroo Flat,4.4,"250 Allingham Street, Kangaroo Flat",['A nice off leash dog park with all basic ame...,"[""park"", ""point_of_interest"", ""establishment""]",,"['Spacious Off-Leash Areas', 'Separate Play Zo...",1,0,...,1,0,1,0,-36.800189,144.249088,131.689415,0.000000,Allingham Reserve Dog Park ['Spacious Off-Leas...,0.114688
255,Harcourt Dog Park,Strathdale,4.7,"204 Crook Street, Strathdale","[""We love this place! It has 3 different dams ...","[""tourist_attraction"", ""park"", ""point_of_inter...",,"['Dog Friendly', 'Water Features', 'Spacious a...",1,0,...,1,1,1,0,-36.756095,144.314293,133.538764,0.000000,"Harcourt Dog Park ['Dog Friendly', 'Water Feat...",0.167204
259,Truscott Reserve Dog Park,California Gully,4.3,"Cnr of Murdock St and, Turner Street, Californ...","[""Nice big park, there's a small dog area aswe...","[""park"", ""point_of_interest"", ""establishment""]",,"['Spacious environment for dogs', 'Separate ar...",1,0,...,1,0,1,0,-36.728133,144.252898,138.686060,0.000000,Truscott Reserve Dog Park ['Spacious environme...,0.157825


In [31]:
def recommeder_v5(user_lat, user_lon, input_select, f_dist, wheelchair_accessible_entrance, food, serves_vegetarian_food, off_leash, dog_water, fenced_off, seating, parking, night_light, shade, wildlife, open_field, alcohol, keyword):
    
    filtered_locs = []
    weights ={
        'prox_score': 0.3,
        'cosine_similarity': 0.7
    }
    results = keyword_search(input_select, keyword)
    
    results['h_dist'] = results.apply(calulate_dist, axis=1, user_lat=user_lat, user_lon=user_lon)
    results['prox_score'] = results['h_dist'].apply(lambda d: max(0, 1-d/f_dist) if f_dist > 0 else 0)
    
    user_reqs = {
        'wheelchair_accessible_entrance': wheelchair_accessible_entrance,
        'food': food, 
        'serves_vegetarian_food': serves_vegetarian_food,
        'off_leash': off_leash, 
        'dog_water': dog_water, 
        'fenced_off': fenced_off, 
        'seating': seating, 
        'parking': parking, 
        'night_light': night_light, 
        'shade': shade, 
        'wildlife': wildlife, 
        'open_field': open_field, 
        'alcohol': alcohol
    }
    
    mask = pd.Series([True] * len(input_select))
    for col, req in user_reqs.items():
        if req == 1:
            mask &= results[col] == 1
    filtered_locs = results[mask]
    
    filtered_locs = filtered_locs[filtered_locs['h_dist'] <= f_dist]
    return filtered_locs

In [32]:
locations = recommeder_v5(-37.847039, 144.945525, input_select, 1000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'dog')

C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['name'] = input_select['name'].fillna('')
C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['description'] = input_select['description'].fillna('')
C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [33]:
locations

,name,suburb,rating,vicinity,review_text,type,editorial_summary.overview,description,wheelchair_accessible_entrance,food,...,shade,wildlife,open_field,alcohol,geometry.location.lat,geometry.location.lng,h_dist,prox_score,full_text,cosine_similarity
0,St Kilda Harbour Dog Beach,St Kilda,4.9,NaN,['Small area of beach just north of the main p...,"[""park"", ""point_of_interest"", ""establishment""]",,"['Dog-friendly beach', 'Scenic views of the ha...",0,1,...,0,1,1,0,-37.860646,144.969802,2.613904,0.997386,St Kilda Harbour Dog Beach ['Dog-friendly beac...,0.125715
1,Cummings Reserve,St Kilda West,4.6,"Cummings Reserve, 299-312 Beaconsfield Parade,...","[""Cummings Reserve is quite a large tapering s...","[""park"", ""point_of_interest"", ""establishment""]",,"['Grassed Area', 'Scenic Sunset Views', 'Dog W...",1,0,...,1,1,1,0,-37.856424,144.965946,2.074537,0.997925,"Cummings Reserve ['Grassed Area', 'Scenic Suns...",0.056770
3,Phillips Reserve,Brunswick East,4.5,Brunswick East,['Council should maintain there pathways they ...,"[""park"", ""point_of_interest"", ""establishment""]",,"['Family-friendly playground', 'Tranquil natur...",1,0,...,1,1,1,0,-37.770638,144.984935,9.173807,0.990826,Phillips Reserve ['Family-friendly playground'...,0.058575
4,Manningham Reserve,Parkville,4.6,"Substation 100/87 Manningham Street, Parkville","[""Nice and clean, it's connected to the soccer...","[""park"", ""point_of_interest"", ""establishment""]",,"['Family-Friendly', 'Sports Activities', 'Gree...",1,0,...,1,1,1,0,-37.785144,144.941920,6.889736,0.993110,"Manningham Reserve ['Family-Friendly', 'Sports...",0.090088
10,Bell Street Coffee Window,Fitzroy,4.4,"233 Brunswick Street, Fitzroy","[""To my taste, the coffee was burnt, not my fa...","[""cafe"", ""bar"", ""food"", ""point_of_interest"", ""...",,"['Coffee Quality', 'Service Experience', 'Comm...",1,1,...,1,0,0,0,-37.800440,144.977879,5.909643,0.994090,"Bell Street Coffee Window ['Coffee Quality', '...",0.029724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Mildura Fenced Dog Park (Rio Vista Park),Mildura,4.2,"Hugh King Drive, Mildura","[""Awesome place to let your dog off lead, the ...","[""park"", ""point_of_interest"", ""establishment""]",,"['Spacious and well-maintained dog park', 'Sha...",1,0,...,1,1,1,0,-34.175648,142.160300,478.915938,0.521084,Mildura Fenced Dog Park (Rio Vista Park) ['Spa...,0.197250
254,Allingham Reserve Dog Park,Kangaroo Flat,4.4,"250 Allingham Street, Kangaroo Flat",['A nice off leash dog park with all basic ame...,"[""park"", ""point_of_interest"", ""establishment""]",,"['Spacious Off-Leash Areas', 'Separate Play Zo...",1,0,...,1,0,1,0,-36.800189,144.249088,131.689415,0.868311,Allingham Reserve Dog Park ['Spacious Off-Leas...,0.114688
255,Harcourt Dog Park,Strathdale,4.7,"204 Crook Street, Strathdale","[""We love this place! It has 3 different dams ...","[""tourist_attraction"", ""park"", ""point_of_inter...",,"['Dog Friendly', 'Water Features', 'Spacious a...",1,0,...,1,1,1,0,-36.756095,144.314293,133.538764,0.866461,"Harcourt Dog Park ['Dog Friendly', 'Water Feat...",0.167204
259,Truscott Reserve Dog Park,California Gully,4.3,"Cnr of Murdock St and, Turner Street, Californ...","[""Nice big park, there's a small dog area aswe...","[""park"", ""point_of_interest"", ""establishment""]",,"['Spacious environment for dogs', 'Separate ar...",1,0,...,1,0,1,0,-36.728133,144.252898,138.686060,0.861314,Truscott Reserve Dog Park ['Spacious environme...,0.157825


In [34]:
def recommeder_final(user_lat, user_lon, input_select, f_dist, wheelchair_accessible_entrance, food, serves_vegetarian_food, off_leash, dog_water, fenced_off, seating, parking, night_light, shade, wildlife, open_field, alcohol, keyword):
    
    filtered_locs = []
    weights ={
        'prox_score': 0.3,
        'cosine_similarity': 0.7
    }
    results = keyword_search(input_select, keyword)
    
    results['h_dist'] = results.apply(calulate_dist, axis=1, user_lat=user_lat, user_lon=user_lon)
    results['prox_score'] = results['h_dist'].apply(lambda d: max(0, 1-d/f_dist) if f_dist > 0 else 0)
    
    user_reqs = {
        'wheelchair_accessible_entrance': wheelchair_accessible_entrance,
        'food': food, 
        'serves_vegetarian_food': serves_vegetarian_food,
        'off_leash': off_leash, 
        'dog_water': dog_water, 
        'fenced_off': fenced_off, 
        'seating': seating, 
        'parking': parking, 
        'night_light': night_light, 
        'shade': shade, 
        'wildlife': wildlife, 
        'open_field': open_field, 
        'alcohol': alcohol
    }
    
    mask = pd.Series([True] * len(input_select))
    for col, req in user_reqs.items():
        if req == 1:
            mask &= results[col] == 1
    filtered_locs = results[mask]
    filtered_locs['weighted_score'] = (filtered_locs['prox_score'] * weights['prox_score'] + filtered_locs['cosine_similarity'] * weights['cosine_similarity'])
    filtered_locs = filtered_locs[filtered_locs['h_dist'] <= f_dist]
    sorted_locs = filtered_locs.sort_values(by='weighted_score', ascending=False)
    display_locs = sorted_locs[['name', 'rating', 'suburb', 'h_dist', 'description']]
    return display_locs.head(10)

In [35]:
# Example One: Busy day in the city with a small dog, require a dog-friendly cafe on your walk together.
example1 = recommeder_final(-37.8124, 144.9623, input_select, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 'dog-friendly cafe')

C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['name'] = input_select['name'].fillna('')
C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['description'] = input_select['description'].fillna('')
C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [36]:
example1

,name,rating,suburb,h_dist,description
17,Café Felice,4.6,Melbourne,0.394666,"['Best coffee in Melbourne CBD', 'Friendly and..."
14,Hot Poppy cafe,4.4,North Melbourne,1.435029,"['Charming Atmosphere', 'Quality Organic Ingre..."
31,Auction Rooms,4.4,North Melbourne,1.574278,"['Rustic-chic decor', 'Sunny courtyard seating..."
10,Bell Street Coffee Window,4.4,Fitzroy,1.908336,"['Coffee Quality', 'Service Experience', 'Comm..."


In [37]:
# Example Two: Living in the eastern suburbs area (Box Hill), looking to explore somewhere with wildlife and open space.
# Since it is the weekend you are willing to spend more time driving.
example2 = recommeder_final(-37.8181, 145.1239, input_select, 30, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 'natural beauty')

C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['name'] = input_select['name'].fillna('')
C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_select['description'] = input_select['description'].fillna('')
C:\Users\liamg\AppData\Local\Temp\ipykernel_4028\1249630063.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [38]:
example2

,name,rating,suburb,h_dist,description
91,Boronia Grove Reserve Dog Off Leash Area,4.3,Doncaster East,3.147982,"['Spacious grassy area for dogs', 'Quiet despi..."
106,Nettleton Park Dog Off Leash Area,4.3,Glen Iris,7.027979,"['Dog-Friendly', 'Family-Friendly', 'Socializa..."
81,Banksia Park Fenced Dog Park,4.4,Bulleen,7.992896,"['Fully fenced for off-lead play', 'Plenty of ..."
3,Phillips Reserve,4.5,Brunswick East,13.302307,"['Family-friendly playground', 'Tranquil natur..."
2,Port Melbourne Beach,4.5,Port Melbourne,15.992423,"['Stunning beauty', 'Family-friendly', 'Gentle..."
111,Nortons Park,4.5,Wantirna South,9.409527,"['Dog-friendly', 'Picnic area', 'Scenic views'..."
83,Rosanna Parklands,4.5,Rosanna,10.018992,"['Nature-friendly urban park', 'Ideal for dog ..."
131,Packer Park Dog Off Leash Area,4.4,Carnegie,10.772727,"['Dog-friendly environment', 'Scenic pond and ..."
125,David Cooper Park,4.4,Wantirna South,10.841162,"['Biking Friendly', 'Playground and BBQ Facili..."
93,Warrandyte River Reserve Dog Off Leash Area,4.6,Warrandyte,11.977229,"['Dog-friendly', 'Scenic walking paths', 'Vibr..."


In [39]:
input_select

,name,suburb,rating,vicinity,review_text,type,editorial_summary.overview,description,wheelchair_accessible_entrance,food,...,shade,wildlife,open_field,alcohol,geometry.location.lat,geometry.location.lng,h_dist,prox_score,full_text,cosine_similarity
0,St Kilda Harbour Dog Beach,St Kilda,4.9,NaN,['Small area of beach just north of the main p...,"[""park"", ""point_of_interest"", ""establishment""]",,"['Dog-friendly beach', 'Scenic views of the ha...",0,1,...,0,1,1,0,-37.860646,144.969802,2.613904,0.947722,St Kilda Harbour Dog Beach ['Dog-friendly beac...,0.000000
1,Cummings Reserve,St Kilda West,4.6,"Cummings Reserve, 299-312 Beaconsfield Parade,...","[""Cummings Reserve is quite a large tapering s...","[""park"", ""point_of_interest"", ""establishment""]",,"['Grassed Area', 'Scenic Sunset Views', 'Dog W...",1,0,...,1,1,1,0,-37.856424,144.965946,2.074537,0.958509,"Cummings Reserve ['Grassed Area', 'Scenic Suns...",0.000000
2,Port Melbourne Beach,Port Melbourne,4.5,Port Melbourne Beach,"['Stunning beauty, You can try fishing at the ...","[""natural_feature"", ""establishment""]",Family-friendly beach with a small wooden fish...,"['Stunning beauty', 'Family-friendly', 'Gentle...",0,1,...,1,1,0,0,-37.847039,144.945525,0.000028,0.999999,"Port Melbourne Beach ['Stunning beauty', 'Fami...",0.107866
3,Phillips Reserve,Brunswick East,4.5,Brunswick East,['Council should maintain there pathways they ...,"[""park"", ""point_of_interest"", ""establishment""]",,"['Family-friendly playground', 'Tranquil natur...",1,0,...,1,1,1,0,-37.770638,144.984935,9.173807,0.816524,Phillips Reserve ['Family-friendly playground'...,0.080389
4,Manningham Reserve,Parkville,4.6,"Substation 100/87 Manningham Street, Parkville","[""Nice and clean, it's connected to the soccer...","[""park"", ""point_of_interest"", ""establishment""]",,"['Family-Friendly', 'Sports Activities', 'Gree...",1,0,...,1,1,1,0,-37.785144,144.941920,6.889736,0.862205,"Manningham Reserve ['Family-Friendly', 'Sports...",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,Strathdale Park Play Space,Kennington,4.6,"148 Crook Street, Kennington",['Beautiful space for everyone to enjoy.\nLove...,"[""park"", ""point_of_interest"", ""establishment""]",,"['Family-friendly', 'Playgrounds', 'Water feat...",1,0,...,1,0,1,0,-36.760569,144.313850,133.102746,0.000000,"Strathdale Park Play Space ['Family-friendly',...",0.000000
258,Rosalind Park,Bendigo,4.6,"30 Gaol Road, Bendigo",['Very nice leafy park with gardens and a cons...,"[""park"", ""tourist_attraction"", ""point_of_inter...","Downtown park with a fernery, an 1800s conserv...","['Leafy park with gardens and conservatory', '...",1,1,...,1,1,0,0,-36.757165,144.278365,134.790979,0.000000,Rosalind Park ['Leafy park with gardens and co...,0.000000
259,Truscott Reserve Dog Park,California Gully,4.3,"Cnr of Murdock St and, Turner Street, Californ...","[""Nice big park, there's a small dog area aswe...","[""park"", ""point_of_interest"", ""establishment""]",,"['Spacious environment for dogs', 'Separate ar...",1,0,...,1,0,1,0,-36.728133,144.252898,138.686060,0.000000,Truscott Reserve Dog Park ['Spacious environme...,0.000000
260,Kennington Reservoir parkrun,Strathdale,4.4,"55 Reservoir Road, Strathdale","['I really like this parkrun, although the dou...","[""park"", ""health"", ""point_of_interest"", ""estab...",,"['Scenic Reservoir Views', 'Friendly Community...",1,0,...,1,1,1,0,-36.772869,144.311283,131.956997,0.000000,Kennington Reservoir parkrun ['Scenic Reservoi...,0.000000
